In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 
import json
from pandas.io.json import json_normalize

In [2]:
os.chdir("GoogleAnalyticsSales/")

In [3]:
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
# converter applies a converter function while you are loading the data, you are loading it as
# json.loadstring = json.loads() for each of those columns in the list
train_set = pd.read_csv("train.csv", 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'})

In [27]:
train_set.groupby("channelGrouping").size()

channelGrouping
(Other)              120
Affiliates         16403
Direct            143026
Display             6262
Organic Search    381561
Paid Search        25326
Referral          104838
Social            226117
dtype: int64

In [ ]:
train_set.groupby("socialEngagementType").size()

In [ ]:
train_set.groupby("visitNumber").size()

In [7]:
# drop it since it has just one value for all the columns
train_set.drop(["socialEngagementType"], axis=1, inplace=True)

In [8]:
# for each column, since you are already loaded it as JSON
for column in JSON_COLUMNS:
        # normalize them, that is whether all the rows has this variable or not, create a seperate column
        # for each of them and give NaN is that attribute is not present for the row
        column_as_df = json_normalize(list(train_set[column]))
        # convert each json key in that column to column.subcolumn, for example, device.isMobile.
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        # drop the main column, for example "device" and merge all the columns you just created like "device.browser"
        train_set = train_set.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
print(train_set.shape, "\n", train_set.columns)

(903653, 54) 
 Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser', 'device.browserSize',
       'device.browserVersion', 'device.deviceCategory', 'device.flashVersion',
       'device.isMobile', 'device.language', 'device.mobileDeviceBranding',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.operatingSystem', 'device.operatingSystemVersion',
       'device.screenColors', 'device.screenResolution', 'geoNetwork.city',
       'geoNetwork.cityId', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.networkLocation',
       'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.transactionRevenue', 'totals.visit

In [ ]:
# go through all the columns in the train_set
for column in train_set.columns:
    # whichever column was a JSON and you split it
    if "." in column:
        print(train_set.groupby(column).size())
        # check how many different values you have in them, if its just one value throughout the dataset, drop it
        if (len(train_set.groupby(column).size()) == 1):
            train_set.drop(column, axis=1, inplace=True)

In [30]:
# you are left with 31 columns
train_set.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device.browser',
       'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.hits', 'totals.pageviews',
       'totals.transactionRevenue', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.keyword', 'trafficSource.medium',
       'trafficSource.referralPath', 'trafficSource.source'],
      dtype='object')

In [42]:
# convert the int64 datatype of date to meaningful dates of typedatetime64
train_set.date = pd.to_datetime(train_set.date,format='%Y%m%d')

In [43]:
train_set.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,...,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,2016-09-02,1.13166E+18,1131660440785968503_1472830385,1472830385,1,1472830385,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google
1,Organic Search,2016-09-02,3.77306E+17,377306020877927890_1472880147,1472880147,1,1472880147,Firefox,desktop,False,...,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google
2,Organic Search,2016-09-02,3.89555E+18,3895546263509774583_1472865386,1472865386,1,1472865386,Chrome,desktop,False,...,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google
3,Organic Search,2016-09-02,4.76345E+18,4763447161404445595_1472881213,1472881213,1,1472881213,UC Browser,desktop,False,...,NaN,NaN,NaN,NaN,NaN,(not set),google + online,organic,NaN,google
4,Organic Search,2016-09-02,2.72944E+16,27294437909732085_1472822600,1472822600,2,1472822600,Chrome,mobile,True,...,NaN,NaN,NaN,NaN,NaN,(not set),(not provided),organic,NaN,google
